In [13]:
import os
import pandas as pd
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [14]:
location=r'E:\Thesis\HMDB/data/train'
classes=[]
videos=[]
train=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
train['video_name']=videos
print(classes)
train.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
1,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
2,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
3,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...
4,brush_hair/Aussie_Brunette_Brushing_Hair_II_br...


In [15]:
location=r'E:\Thesis\HMDB/data/test'
classes=[]
videos=[]
test=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
test['video_name']=videos
print(classes)
test.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
1,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
2,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
3,brush_hair/Blonde_being_brushed_brush_hair_f_n...
4,brush_hair/Blonde_being_brushed_brush_hair_u_c...


In [16]:
location=r'E:\Thesis\HMDB/data/val'
classes=[]
videos=[]
val=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
val['video_name']=videos
print(classes)
val.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UP...
1,brush_hair/Brushing_hair__the_right_way_brush_...
2,brush_hair/Brushing_my_Long_Hair__February_200...
3,brush_hair/Ella_brushing_her_amazing_long_hair...
4,brush_hair/Haarek_mmen_brush_hair_h_cm_np1_fr_...


In [17]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

In [18]:
# creating tags for test videos
val_video_tag = []
for i in range(val.shape[0]):
    val_video_tag.append(val['video_name'][i].split('/')[0])
    
val['tag'] = val_video_tag

In [19]:
# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [20]:
train_data = pd.DataFrame(columns=['image','class'])
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    classname=videoFile.split(' ')[0].split('/')[0]
#    print('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture(r'E:\Thesis\HMDB/data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    # print(frameRate)
    ret, frame1 = cap.read()
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step=length/9
    # print('step')
    # print(step)
#    print('length')
#    print(length)
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    x=1
    while(x<length):
#        print(x)
        frameId = cap.get(1) #current frame number
        ret, frame2 = cap.read()
        x+=1
        if (ret != True):
            break
        if (frameId % math.floor(step) == 0):
            if(count>7):
                # print(count)
                break
            name=(videoFile.split('/')[1]).split('.')[0];
            filename =r'E:\Thesis\HMDB/data/train_frame_OF_8/' + name +"_flow%d.jpg" % count;
            imgname=name +"_flow%d.jpg" % count  
            count+=1          
#            print(filename)
            exist=os.path.exists(filename)
            if exist==False:
                # storing the frames in a new folder named train_1
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                cv2.imwrite(filename, rgb)
#            print(filename)
#            print(imgname)  
#            print(classname)
            train_data=train_data.append({'image':imgname,'class':classname},ignore_index=True)
    
    if(count<8):
        print('frames written')
        print(count)
    cap.release()





  0%|          | 0/3570 [00:00<?, ?it/s]



  0%|          | 1/3570 [00:02<2:03:49,  2.08s/it]



  0%|          | 2/3570 [00:03<1:46:34,  1.79s/it]



  0%|          | 3/3570 [00:05<1:53:11,  1.90s/it]



  0%|          | 4/3570 [00:08<2:12:44,  2.23s/it]



  0%|          | 5/3570 [00:09<2:01:08,  2.04s/it]



  0%|          | 6/3570 [00:12<2:11:46,  2.22s/it]



  0%|          | 7/3570 [00:14<2:04:15,  2.09s/it]



  0%|          | 8/3570 [00:17<2:14:42,  2.27s/it]



  0%|          | 9/3570 [00:18<2:02:26,  2.06s/it]



  0%|          | 10/3570 [00:21<2:23:39,  2.42s/it]



  0%|          | 11/3570 [00:23<2:16:32,  2.30s/it]



  0%|          | 12/3570 [00:25<1:56:48,  1.97s/it]



  0%|          | 13/3570 [00:27<1:58:57,  2.01s/it]



  0%|          | 14/3570 [00:30<2:21:35,  2.39s/it]



  0%|          | 15/3570 [00:31<2:00:30,  2.03s/it]



  0%|          | 16/3570 [00:34<2:06:59,  2.14s/it]



  0%|          | 17/3570 [00:35<1:59:07,  2.01s/it]



  1%|          | 18/3570 [

  8%|▊         | 294/3570 [08:37<1:34:09,  1.72s/it]



  8%|▊         | 295/3570 [08:39<1:30:16,  1.65s/it]



  8%|▊         | 296/3570 [08:41<1:40:46,  1.85s/it]



  8%|▊         | 297/3570 [08:43<1:34:56,  1.74s/it]



  8%|▊         | 298/3570 [08:44<1:32:05,  1.69s/it]



  8%|▊         | 299/3570 [08:46<1:30:36,  1.66s/it]



  8%|▊         | 300/3570 [08:48<1:37:26,  1.79s/it]



  8%|▊         | 301/3570 [08:49<1:33:31,  1.72s/it]



  8%|▊         | 302/3570 [08:51<1:31:23,  1.68s/it]



  8%|▊         | 303/3570 [08:53<1:29:51,  1.65s/it]



  9%|▊         | 304/3570 [08:54<1:32:19,  1.70s/it]



  9%|▊         | 305/3570 [08:56<1:28:21,  1.62s/it]



  9%|▊         | 306/3570 [08:58<1:41:31,  1.87s/it]



  9%|▊         | 307/3570 [09:00<1:41:42,  1.87s/it]



  9%|▊         | 308/3570 [09:03<1:57:06,  2.15s/it]



  9%|▊         | 309/3570 [09:06<2:09:24,  2.38s/it]



  9%|▊         | 310/3570 [09:08<2:04:23,  2.29s/it]



  9%|▊         | 311/3570 [09:11<2:12:26,  2.44s

 16%|█▋        | 586/3570 [18:52<1:45:30,  2.12s/it]



 16%|█▋        | 587/3570 [18:53<1:29:12,  1.79s/it]



 16%|█▋        | 588/3570 [18:55<1:30:37,  1.82s/it]



 16%|█▋        | 589/3570 [18:58<1:36:48,  1.95s/it]



 17%|█▋        | 590/3570 [19:00<1:50:52,  2.23s/it]



 17%|█▋        | 591/3570 [19:03<1:49:11,  2.20s/it]



 17%|█▋        | 592/3570 [19:04<1:41:26,  2.04s/it]



 17%|█▋        | 593/3570 [19:06<1:35:28,  1.92s/it]



 17%|█▋        | 594/3570 [19:08<1:31:22,  1.84s/it]



 17%|█▋        | 595/3570 [19:11<1:50:20,  2.23s/it]



 17%|█▋        | 596/3570 [19:12<1:38:14,  1.98s/it]



 17%|█▋        | 597/3570 [19:14<1:33:16,  1.88s/it]



 17%|█▋        | 598/3570 [19:16<1:38:08,  1.98s/it]



 17%|█▋        | 599/3570 [19:18<1:39:39,  2.01s/it]



 17%|█▋        | 600/3570 [19:20<1:43:39,  2.09s/it]



 17%|█▋        | 601/3570 [19:22<1:41:49,  2.06s/it]



 17%|█▋        | 602/3570 [19:25<1:51:25,  2.25s/it]



 17%|█▋        | 603/3570 [19:27<1:44:58,  2.12s

 25%|██▍       | 878/3570 [31:01<1:29:20,  1.99s/it]



 25%|██▍       | 879/3570 [31:03<1:25:24,  1.90s/it]



 25%|██▍       | 880/3570 [31:05<1:22:45,  1.85s/it]



 25%|██▍       | 881/3570 [31:07<1:26:35,  1.93s/it]



 25%|██▍       | 882/3570 [31:09<1:25:50,  1.92s/it]



 25%|██▍       | 883/3570 [31:11<1:29:16,  1.99s/it]



 25%|██▍       | 884/3570 [31:12<1:15:28,  1.69s/it]



 25%|██▍       | 885/3570 [31:13<1:08:09,  1.52s/it]



 25%|██▍       | 886/3570 [31:15<1:13:46,  1.65s/it]



 25%|██▍       | 887/3570 [31:17<1:16:03,  1.70s/it]



 25%|██▍       | 888/3570 [31:19<1:18:07,  1.75s/it]



 25%|██▍       | 889/3570 [31:20<1:12:37,  1.63s/it]



 25%|██▍       | 890/3570 [31:22<1:23:07,  1.86s/it]



 25%|██▍       | 891/3570 [31:24<1:21:01,  1.81s/it]



 25%|██▍       | 892/3570 [31:26<1:27:05,  1.95s/it]



 25%|██▌       | 893/3570 [31:28<1:26:57,  1.95s/it]



 25%|██▌       | 894/3570 [31:30<1:24:34,  1.90s/it]



 25%|██▌       | 895/3570 [31:32<1:20:53,  1.81s

 33%|███▎      | 1166/3570 [39:22<1:15:14,  1.88s/it]



 33%|███▎      | 1167/3570 [39:24<1:18:36,  1.96s/it]



 33%|███▎      | 1168/3570 [39:26<1:20:04,  2.00s/it]



 33%|███▎      | 1169/3570 [39:28<1:17:12,  1.93s/it]



 33%|███▎      | 1170/3570 [39:30<1:26:28,  2.16s/it]



 33%|███▎      | 1171/3570 [39:32<1:24:39,  2.12s/it]



 33%|███▎      | 1172/3570 [39:35<1:24:46,  2.12s/it]



 33%|███▎      | 1173/3570 [39:36<1:18:41,  1.97s/it]



 33%|███▎      | 1174/3570 [39:38<1:11:49,  1.80s/it]



 33%|███▎      | 1175/3570 [39:39<1:09:05,  1.73s/it]



 33%|███▎      | 1176/3570 [39:41<1:11:07,  1.78s/it]



 33%|███▎      | 1177/3570 [39:43<1:14:36,  1.87s/it]



 33%|███▎      | 1178/3570 [39:45<1:14:24,  1.87s/it]



 33%|███▎      | 1179/3570 [39:47<1:10:33,  1.77s/it]



 33%|███▎      | 1180/3570 [39:48<1:08:19,  1.72s/it]



 33%|███▎      | 1181/3570 [39:49<1:04:06,  1.61s/it]



 33%|███▎      | 1182/3570 [39:51<1:01:54,  1.56s/it]



 33%|███▎      | 1183/3570 [39:

...

 62%|██████▏   | 2202/3570 [1:01:25<53:12,  2.33s/it]



 62%|██████▏   | 2203/3570 [1:01:28<54:30,  2.39s/it]



 62%|██████▏   | 2204/3570 [1:01:30<51:43,  2.27s/it]



 62%|██████▏   | 2205/3570 [1:01:32<53:19,  2.34s/it]



 62%|██████▏   | 2206/3570 [1:01:34<51:28,  2.26s/it]



 62%|██████▏   | 2207/3570 [1:01:36<46:39,  2.05s/it]



 62%|██████▏   | 2208/3570 [1:01:38<49:45,  2.19s/it]



 62%|██████▏   | 2209/3570 [1:01:40<45:12,  1.99s/it]



 62%|██████▏   | 2210/3570 [1:01:42<50:05,  2.21s/it]



 62%|██████▏   | 2211/3570 [1:01:45<52:58,  2.34s/it]



 62%|██████▏   | 2212/3570 [1:01:47<47:45,  2.11s/it]



 62%|██████▏   | 2213/3570 [1:01:49<48:07,  2.13s/it]



 62%|██████▏   | 2214/3570 [1:01:50<41:56,  1.86s/it]



 62%|██████▏   | 2215/3570 [1:01:52<43:10,  1.91s/it]



 62%|██████▏   | 2216/3570 [1:01:54<45:26,  2.01s/it]



 62%|██████▏   | 2217/3570 [1:01:56<42:15,  1.87s/it]



 62%|██████▏   | 2218/3570 [1:01:57<38:36,  1.71s/it]



 62%|██████▏   | 2219/3570 [1:0

 70%|██████▉   | 2488/3570 [1:10:18<44:06,  2.45s/it]



 70%|██████▉   | 2489/3570 [1:10:20<42:54,  2.38s/it]



 70%|██████▉   | 2490/3570 [1:10:22<42:10,  2.34s/it]



 70%|██████▉   | 2491/3570 [1:10:25<43:32,  2.42s/it]



 70%|██████▉   | 2492/3570 [1:10:28<49:15,  2.74s/it]



 70%|██████▉   | 2493/3570 [1:10:30<45:46,  2.55s/it]



 70%|██████▉   | 2494/3570 [1:10:32<38:40,  2.16s/it]



 70%|██████▉   | 2495/3570 [1:10:33<33:56,  1.89s/it]



 70%|██████▉   | 2496/3570 [1:10:35<35:21,  1.98s/it]



 70%|██████▉   | 2497/3570 [1:10:38<42:12,  2.36s/it]



 70%|██████▉   | 2498/3570 [1:10:40<36:17,  2.03s/it]



 70%|███████   | 2499/3570 [1:10:41<33:23,  1.87s/it]



 70%|███████   | 2500/3570 [1:10:43<30:27,  1.71s/it]



 70%|███████   | 2501/3570 [1:10:44<28:14,  1.59s/it]



 70%|███████   | 2502/3570 [1:10:46<30:25,  1.71s/it]



 70%|███████   | 2503/3570 [1:14:25<19:53:11, 67.10s/it]



 70%|███████   | 2504/3570 [1:14:27<14:02:39, 47.43s/it]



 70%|███████   | 2505/357

 78%|███████▊  | 2774/3570 [1:18:56<09:16,  1.43it/s]



 78%|███████▊  | 2775/3570 [1:18:57<10:01,  1.32it/s]



 78%|███████▊  | 2776/3570 [1:18:57<08:53,  1.49it/s]



 78%|███████▊  | 2777/3570 [1:18:58<10:05,  1.31it/s]



 78%|███████▊  | 2778/3570 [1:18:59<09:29,  1.39it/s]



 78%|███████▊  | 2779/3570 [1:19:00<10:17,  1.28it/s]



 78%|███████▊  | 2780/3570 [1:19:01<11:26,  1.15it/s]



 78%|███████▊  | 2781/3570 [1:19:02<10:49,  1.21it/s]



 78%|███████▊  | 2782/3570 [1:19:02<10:11,  1.29it/s]



 78%|███████▊  | 2783/3570 [1:19:03<10:21,  1.27it/s]



 78%|███████▊  | 2784/3570 [1:19:04<09:34,  1.37it/s]



 78%|███████▊  | 2785/3570 [1:19:04<10:15,  1.28it/s]



 78%|███████▊  | 2786/3570 [1:19:05<10:08,  1.29it/s]



 78%|███████▊  | 2787/3570 [1:19:06<09:57,  1.31it/s]



 78%|███████▊  | 2788/3570 [1:19:07<09:44,  1.34it/s]



 78%|███████▊  | 2789/3570 [1:19:08<10:28,  1.24it/s]



 78%|███████▊  | 2790/3570 [1:19:08<10:22,  1.25it/s]



 78%|███████▊  | 2791/3570 [1:1

 86%|████████▌ | 3060/3570 [1:23:33<07:10,  1.19it/s]



 86%|████████▌ | 3061/3570 [1:23:34<07:41,  1.10it/s]



 86%|████████▌ | 3062/3570 [1:23:35<07:55,  1.07it/s]



 86%|████████▌ | 3063/3570 [1:23:36<07:46,  1.09it/s]



 86%|████████▌ | 3064/3570 [1:23:37<08:28,  1.01s/it]



 86%|████████▌ | 3065/3570 [1:23:38<07:52,  1.07it/s]



 86%|████████▌ | 3066/3570 [1:23:39<09:01,  1.07s/it]



 86%|████████▌ | 3067/3570 [1:23:40<08:35,  1.02s/it]



 86%|████████▌ | 3068/3570 [1:23:41<08:07,  1.03it/s]



 86%|████████▌ | 3069/3570 [1:23:42<07:33,  1.10it/s]



 86%|████████▌ | 3070/3570 [1:23:43<06:55,  1.20it/s]



 86%|████████▌ | 3071/3570 [1:23:43<07:08,  1.16it/s]



 86%|████████▌ | 3072/3570 [1:23:44<07:02,  1.18it/s]



 86%|████████▌ | 3073/3570 [1:23:45<06:25,  1.29it/s]



 86%|████████▌ | 3074/3570 [1:23:46<06:38,  1.25it/s]



 86%|████████▌ | 3075/3570 [1:23:46<06:26,  1.28it/s]



 86%|████████▌ | 3076/3570 [1:23:48<07:07,  1.16it/s]



 86%|████████▌ | 3077/3570 [1:2

 94%|█████████▎| 3346/3570 [1:28:02<03:01,  1.23it/s]



 94%|█████████▍| 3347/3570 [1:28:03<03:47,  1.02s/it]



 94%|█████████▍| 3348/3570 [1:28:04<04:02,  1.09s/it]



 94%|█████████▍| 3349/3570 [1:28:06<04:05,  1.11s/it]



 94%|█████████▍| 3350/3570 [1:28:06<03:38,  1.01it/s]



 94%|█████████▍| 3351/3570 [1:28:07<03:29,  1.04it/s]



 94%|█████████▍| 3352/3570 [1:28:08<03:26,  1.06it/s]



 94%|█████████▍| 3353/3570 [1:28:09<03:16,  1.10it/s]



 94%|█████████▍| 3354/3570 [1:28:10<02:59,  1.20it/s]



 94%|█████████▍| 3355/3570 [1:28:11<03:13,  1.11it/s]



 94%|█████████▍| 3356/3570 [1:28:11<03:02,  1.17it/s]



 94%|█████████▍| 3357/3570 [1:28:12<02:58,  1.20it/s]



 94%|█████████▍| 3358/3570 [1:28:13<02:45,  1.28it/s]



 94%|█████████▍| 3359/3570 [1:28:14<03:44,  1.06s/it]



 94%|█████████▍| 3360/3570 [1:28:16<04:10,  1.19s/it]



 94%|█████████▍| 3361/3570 [1:28:17<04:11,  1.20s/it]



 94%|█████████▍| 3362/3570 [1:28:18<03:38,  1.05s/it]



 94%|█████████▍| 3363/3570 [1:2

In [21]:
train_data.head()

,image,class
0,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
1,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
2,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair


In [22]:
train_data.nunique()

image    28560
class       51
dtype: int64

In [23]:
# converting the dataframe into csv file 
train_data.to_csv('../data/train_OF_8.csv',header=True, index=False)

In [24]:
train = pd.read_csv('../data/train_OF_8.csv')
train.head()

,image,class
0,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
1,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
2,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
3,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair
4,Aussie_Brunette_Brushing_Hair_II_brush_hair_u_...,brush_hair


In [25]:
val_data = pd.DataFrame(columns=['image','class'])
# storing the frames from training videos
for i in tqdm(range(val.shape[0])):
    count = 0
    videoFile = val['video_name'][i]
    classname=videoFile.split(' ')[0].split('/')[0]
#    print('../data/train/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])
    cap = cv2.VideoCapture(r'E:\Thesis\HMDB/data/val/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    # print(frameRate)
    ret, frame1 = cap.read()
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step=length/9
    # print('step')
    # print(step)
#    print('length')
#    print(length)
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    x=1
    while(x<length):
#        print(x)
        frameId = cap.get(1) #current frame number
        ret, frame2 = cap.read()
        x+=1
        if (ret != True):
            break
        if (frameId % math.floor(step) == 0):
            if(count>7):
                # print(count)
                break
            name=(videoFile.split('/')[1]).split('.')[0];
            filename =r'E:\Thesis\HMDB/data/val_frame_OF_8/' + name +"_flow%d.jpg" % count;
            imgname=name +"_flow%d.jpg" % count   
            count+=1         
            train_data=train_data.append({'image':imgname,'class':classname},ignore_index=True)
#            print(filename)
            exist=os.path.exists(filename)
            if exist==False:
                # storing the frames in a new folder named train_1
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                cv2.imwrite(filename, rgb)
#            print(filename)
#            print(imgname)         
            val_data=val_data.append({'image':imgname,'class':classname},ignore_index=True)
    if(count<8):
        print('frames written')
        print(count)
    cap.release()





  0%|          | 0/1666 [00:00<?, ?it/s]



  0%|          | 1/1666 [00:00<20:55,  1.33it/s]



  0%|          | 2/1666 [00:02<29:39,  1.07s/it]



  0%|          | 3/1666 [00:04<33:06,  1.19s/it]



  0%|          | 4/1666 [00:05<31:55,  1.15s/it]



  0%|          | 5/1666 [00:06<31:56,  1.15s/it]



  0%|          | 6/1666 [00:07<31:58,  1.16s/it]



  0%|          | 7/1666 [00:08<28:47,  1.04s/it]



  0%|          | 8/1666 [00:09<27:23,  1.01it/s]



  1%|          | 9/1666 [00:09<23:30,  1.17it/s]



  1%|          | 10/1666 [00:10<21:39,  1.27it/s]



  1%|          | 11/1666 [00:10<18:57,  1.46it/s]



  1%|          | 12/1666 [00:11<20:08,  1.37it/s]



  1%|          | 13/1666 [00:12<20:36,  1.34it/s]



  1%|          | 14/1666 [00:13<23:46,  1.16it/s]



  1%|          | 15/1666 [00:14<23:32,  1.17it/s]



  1%|          | 16/1666 [00:15<23:31,  1.17it/s]



  1%|          | 17/1666 [00:16<26:42,  1.03it/s]



  1%|          | 18/1666 [00:17<25:34,  1.07it/s]



  1%|  

 18%|█▊        | 304/1666 [04:06<13:21,  1.70it/s]



 18%|█▊        | 305/1666 [04:06<14:37,  1.55it/s]



 18%|█▊        | 306/1666 [04:07<14:43,  1.54it/s]



 18%|█▊        | 307/1666 [04:08<14:36,  1.55it/s]



 18%|█▊        | 308/1666 [04:08<14:34,  1.55it/s]



 19%|█▊        | 309/1666 [04:09<15:51,  1.43it/s]



 19%|█▊        | 310/1666 [04:10<14:56,  1.51it/s]



 19%|█▊        | 311/1666 [04:10<14:40,  1.54it/s]



 19%|█▊        | 312/1666 [04:12<20:16,  1.11it/s]



 19%|█▉        | 313/1666 [04:13<19:45,  1.14it/s]



 19%|█▉        | 314/1666 [04:14<19:50,  1.14it/s]



 19%|█▉        | 315/1666 [04:14<17:42,  1.27it/s]



 19%|█▉        | 316/1666 [04:15<19:27,  1.16it/s]



 19%|█▉        | 317/1666 [04:16<17:22,  1.29it/s]



 19%|█▉        | 318/1666 [04:16<16:40,  1.35it/s]



 19%|█▉        | 319/1666 [04:17<17:33,  1.28it/s]



 19%|█▉        | 320/1666 [04:18<17:13,  1.30it/s]



 19%|█▉        | 321/1666 [04:19<18:56,  1.18it/s]



 19%|█▉        | 322/1666 [0

 36%|███▋      | 606/1666 [08:08<14:34,  1.21it/s]



 36%|███▋      | 607/1666 [08:09<13:29,  1.31it/s]



 36%|███▋      | 608/1666 [08:09<13:46,  1.28it/s]



 37%|███▋      | 609/1666 [08:10<12:36,  1.40it/s]



 37%|███▋      | 610/1666 [08:11<12:40,  1.39it/s]



 37%|███▋      | 611/1666 [08:12<13:33,  1.30it/s]



 37%|███▋      | 612/1666 [08:13<15:12,  1.16it/s]



 37%|███▋      | 613/1666 [08:14<18:08,  1.03s/it]



 37%|███▋      | 614/1666 [08:15<19:18,  1.10s/it]



 37%|███▋      | 615/1666 [08:16<17:45,  1.01s/it]



 37%|███▋      | 616/1666 [08:17<16:32,  1.06it/s]



 37%|███▋      | 617/1666 [08:18<16:18,  1.07it/s]



 37%|███▋      | 618/1666 [08:19<17:06,  1.02it/s]



 37%|███▋      | 619/1666 [08:20<16:44,  1.04it/s]



 37%|███▋      | 620/1666 [08:20<13:55,  1.25it/s]



 37%|███▋      | 621/1666 [08:21<14:16,  1.22it/s]



 37%|███▋      | 622/1666 [08:22<13:41,  1.27it/s]



 37%|███▋      | 623/1666 [08:23<17:31,  1.01s/it]



 37%|███▋      | 624/1666 [0

 55%|█████▍    | 908/1666 [12:13<08:35,  1.47it/s]



 55%|█████▍    | 909/1666 [12:14<09:56,  1.27it/s]



 55%|█████▍    | 910/1666 [12:15<11:49,  1.07it/s]



 55%|█████▍    | 911/1666 [12:16<13:20,  1.06s/it]



 55%|█████▍    | 912/1666 [12:18<13:13,  1.05s/it]



 55%|█████▍    | 913/1666 [12:18<11:54,  1.05it/s]



 55%|█████▍    | 914/1666 [12:19<11:12,  1.12it/s]



 55%|█████▍    | 915/1666 [12:20<11:21,  1.10it/s]



 55%|█████▍    | 916/1666 [12:21<11:13,  1.11it/s]



 55%|█████▌    | 917/1666 [12:22<11:18,  1.10it/s]



 55%|█████▌    | 918/1666 [12:23<11:49,  1.05it/s]



 55%|█████▌    | 919/1666 [12:24<12:25,  1.00it/s]



 55%|█████▌    | 920/1666 [12:25<14:27,  1.16s/it]



 55%|█████▌    | 921/1666 [12:26<14:04,  1.13s/it]



 55%|█████▌    | 922/1666 [12:27<12:37,  1.02s/it]



 55%|█████▌    | 923/1666 [12:28<11:28,  1.08it/s]



 55%|█████▌    | 924/1666 [12:29<10:35,  1.17it/s]



 56%|█████▌    | 925/1666 [12:30<11:06,  1.11it/s]



 56%|█████▌    | 926/1666 [1

 72%|███████▏  | 1207/1666 [16:33<06:11,  1.24it/s]



 73%|███████▎  | 1208/1666 [16:33<06:15,  1.22it/s]



 73%|███████▎  | 1209/1666 [16:34<06:38,  1.15it/s]



 73%|███████▎  | 1210/1666 [16:35<06:31,  1.16it/s]



 73%|███████▎  | 1211/1666 [16:36<05:58,  1.27it/s]



 73%|███████▎  | 1212/1666 [16:36<05:15,  1.44it/s]



 73%|███████▎  | 1213/1666 [16:37<04:49,  1.57it/s]



 73%|███████▎  | 1214/1666 [16:37<04:40,  1.61it/s]



 73%|███████▎  | 1215/1666 [16:38<04:33,  1.65it/s]



 73%|███████▎  | 1216/1666 [16:39<05:24,  1.39it/s]



 73%|███████▎  | 1217/1666 [16:40<05:14,  1.43it/s]



 73%|███████▎  | 1218/1666 [16:40<05:28,  1.36it/s]



 73%|███████▎  | 1219/1666 [16:41<04:58,  1.50it/s]



 73%|███████▎  | 1220/1666 [16:42<05:22,  1.38it/s]



 73%|███████▎  | 1221/1666 [16:43<05:30,  1.35it/s]



 73%|███████▎  | 1222/1666 [16:43<05:33,  1.33it/s]



 73%|███████▎  | 1223/1666 [16:45<06:39,  1.11it/s]



 73%|███████▎  | 1224/1666 [16:45<06:29,  1.13it/s]



 74%|█████

 81%|████████▏ | 1356/1666 [18:44<04:11,  1.23it/s]



 81%|████████▏ | 1357/1666 [18:45<04:16,  1.20it/s]



 82%|████████▏ | 1358/1666 [18:46<04:41,  1.09it/s]



 82%|████████▏ | 1359/1666 [18:47<04:51,  1.05it/s]



 82%|████████▏ | 1360/1666 [18:48<04:59,  1.02it/s]



 82%|████████▏ | 1361/1666 [18:48<04:39,  1.09it/s]



 82%|████████▏ | 1362/1666 [18:49<04:40,  1.08it/s]



 82%|████████▏ | 1363/1666 [18:50<04:34,  1.10it/s]



 82%|████████▏ | 1364/1666 [18:51<04:23,  1.14it/s]



 82%|████████▏ | 1365/1666 [18:53<05:17,  1.06s/it]



 82%|████████▏ | 1366/1666 [18:54<05:32,  1.11s/it]



 82%|████████▏ | 1367/1666 [18:55<05:12,  1.04s/it]



 82%|████████▏ | 1368/1666 [18:56<05:42,  1.15s/it]



 82%|████████▏ | 1369/1666 [18:58<06:17,  1.27s/it]



 82%|████████▏ | 1370/1666 [18:59<06:00,  1.22s/it]



 82%|████████▏ | 1371/1666 [19:00<05:50,  1.19s/it]



 82%|████████▏ | 1372/1666 [19:01<05:18,  1.08s/it]



 82%|████████▏ | 1373/1666 [19:02<05:03,  1.04s/it]



 82%|█████

 90%|█████████ | 1505/1666 [20:59<02:21,  1.14it/s]



 90%|█████████ | 1506/1666 [20:59<02:11,  1.22it/s]



 90%|█████████ | 1507/1666 [21:01<02:39,  1.00s/it]



 91%|█████████ | 1508/1666 [21:02<02:36,  1.01it/s]



 91%|█████████ | 1509/1666 [21:03<02:41,  1.03s/it]



 91%|█████████ | 1510/1666 [21:04<02:43,  1.05s/it]



 91%|█████████ | 1511/1666 [21:05<02:31,  1.02it/s]



 91%|█████████ | 1512/1666 [21:05<02:15,  1.14it/s]



 91%|█████████ | 1513/1666 [21:06<01:58,  1.29it/s]



 91%|█████████ | 1514/1666 [21:07<01:49,  1.39it/s]



 91%|█████████ | 1515/1666 [21:08<01:59,  1.26it/s]



 91%|█████████ | 1516/1666 [21:08<01:57,  1.28it/s]



 91%|█████████ | 1517/1666 [21:09<01:54,  1.30it/s]



 91%|█████████ | 1518/1666 [21:10<01:41,  1.45it/s]



 91%|█████████ | 1519/1666 [21:11<01:53,  1.29it/s]



 91%|█████████ | 1520/1666 [21:11<01:47,  1.36it/s]



 91%|█████████▏| 1521/1666 [21:13<02:12,  1.09it/s]



 91%|█████████▏| 1522/1666 [21:13<01:55,  1.25it/s]



 91%|█████

 99%|█████████▉| 1654/1666 [23:06<00:09,  1.29it/s]



 99%|█████████▉| 1655/1666 [23:07<00:09,  1.18it/s]



 99%|█████████▉| 1656/1666 [23:08<00:07,  1.29it/s]



 99%|█████████▉| 1657/1666 [23:09<00:07,  1.18it/s]



100%|█████████▉| 1658/1666 [23:10<00:06,  1.27it/s]



100%|█████████▉| 1659/1666 [23:11<00:06,  1.03it/s]



100%|█████████▉| 1660/1666 [23:12<00:05,  1.04it/s]



100%|█████████▉| 1661/1666 [23:13<00:04,  1.02it/s]



100%|█████████▉| 1662/1666 [23:14<00:03,  1.10it/s]



100%|█████████▉| 1663/1666 [23:14<00:02,  1.22it/s]



100%|█████████▉| 1664/1666 [23:15<00:01,  1.27it/s]



100%|█████████▉| 1665/1666 [23:16<00:00,  1.27it/s]



100%|██████████| 1666/1666 [23:17<00:00,  1.14it/s]

In [26]:
val_data.nunique()

image    13328
class       51
dtype: int64

In [27]:
# converting the dataframe into csv file 
val_data.to_csv('../data/val_OF_8.csv',header=True, index=False)

In [28]:
val = pd.read_csv('../data/val_OF_8.csv')
val.head()

,image,class
0,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
1,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
2,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
3,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
4,Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_b...,brush_hair
